----
<img src="../../files/lseg.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Delivery layer - OMM Stream - Level 1 data subscription
This notebook demonstrates how to use an OMM stream of the low level Delivery layer to subscribe to retrieve level 1 streaming data (such as trades and quotes).

#### Learn more

To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
the [Data Library Q&A Forum](https://community.developers.refinitiv.com/smart-spaces/521/index.html). 
The LSEG Developer Community will be happy to help. 

----

## Some Imports to start with

In [5]:
import lseg.data as ld
from lseg.data.delivery import omm_stream
import datetime
import json

## Open the data session

The open_session() function creates and open sessions based on the information contained in the lseg-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [6]:
ld.open_session()

<lseg.data.session.Definition object at 0x1096ae760 {name='workspace'}>

## Retrieve data

### Define a calback to display received events

In [7]:
# Callback function to display data or status events
def display_event(eventType, event):
    currentTime = datetime.datetime.now().time()
    print("----------------------------------------------------------")
    print(">>> {} event received at {}".format(eventType, currentTime))
    print(json.dumps(event, indent=2))
    return

### Create an OMM stream and register event callbacks

In [8]:
stream = omm_stream.Definition(
    name="EUR=", 
    fields=['BID', 'ASK','OPEN_PRC','HST_CLOSE','TIMACT'], 
    domain='MarketPrice').get_stream()

# Refresh - the first full image we get back from the server
stream.on_refresh(lambda event, item_stream  : display_event("Refresh", event))

# Update - as and when field values change, we receive updates from the server
stream.on_update(lambda event, item_stream : display_event("Update", event))

# Status - if data goes stale or item closes, we get a status message
stream.on_status(lambda event, item_stream : display_event("Status", event))

# Other errors
stream.on_error(lambda event, item_stream : display_event("Error", event))

### Open the stream
This example uses the open() synchronous call to open the stream. Because it is a synchronous call, the first notification (either via on_refresh(), on_status() or on_error()) happens before the open() method returns. If the open_async() asynchronous method is used instead, the first notification happens after open_async() returns.

In [9]:
# Send request to server and open stream
stream.open()
# We should receive the initial Refresh for the current field values
# followed by updates for the fields as and when they occur

----------------------------------------------------------
>>> Refresh event received at 09:54:44.614126
{
  "Fields": {
    "ASK": 1.112,
    "BID": 1.1119,
    "HST_CLOSE": 1.111,
    "OPEN_PRC": 1.1109,
    "TIMACT": "08:54:00"
  },
  "State": {
    "Data": "Ok",
    "Stream": "Open",
    "Code": "None",
    "Text": "All is well"
  },
  "Qos": {
    "Rate": "TickByTick",
    "Timeliness": "Realtime"
  },
  "Type": "Refresh",
  "SeqNumber": 63534,
  "ID": 6,
  "Key": {
    "Name": "EUR=",
    "NameType": "Ric",
    "Service": "Q"
  }
}


<OpenState.Opened: 'Opened'>

----------------------------------------------------------
>>> Update event received at 09:54:44.658759
{
  "DoNotRipple": true,
  "Fields": {
    "ASK": 1.112,
    "BID": 1.1119,
    "TIMACT": "08:54:00"
  },
  "Type": "Update",
  "SeqNumber": 63550,
  "ID": 6,
  "Key": {
    "Name": "EUR=",
    "NameType": "Ric",
    "Service": "Q"
  }
}
----------------------------------------------------------
>>> Update event received at 09:54:45.272769
{
  "DoNotRipple": true,
  "Fields": {
    "ASK": 1.112,
    "BID": 1.1119,
    "TIMACT": "08:54:00"
  },
  "Type": "Update",
  "SeqNumber": 63662,
  "ID": 6,
  "Key": {
    "Name": "EUR=",
    "NameType": "Ric",
    "Service": "Q"
  }
}


### Close the stream

In [10]:
stream.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [11]:
ld.close_session()